In [1]:
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
import os
import sys
import pandas as pd
from datetime import date, time
import datetime
import shutil
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader, TextLoader , Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')

In [3]:
pinecone.init(api_key=PINECONE_API_KEY, environment='gcp-starter') # key de Jeremias44

In [4]:
indexName = 'index-skill-matchcv' #indexName de Jeremias44

In [5]:
try:
    if pinecone.list_indexes() == []:
        pinecone.create_index(name=indexName, dimension=384, metric='cosine')
    indexes = pinecone.list_indexes()
    print("Índice creado:", indexes)
except Exception as e:
    print("Error al listar los índices de Pinecone:", str(e))

Índice creado: ['index-skill-matchcv']


In [6]:
# Conexión con index
index = pinecone.Index(index_name=indexName)

In [7]:
print('inicia descarga de modelo de embeddings de HuggingFace')
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
print('modelo descargado')

inicia descarga de modelo de embeddings de HuggingFace
modelo descargado


In [ ]:
#textSplitter = RecursiveCharacterTextSplitter(chunk_size=1500,
#                                              chunk_overlap=200,
#                                              length_function=len,
#                                              #is_separator_regex = False,
#                                              separators=['.\n\n', '.\n', '.', '\n\n', ',', ' ']
#                                               )

In [8]:
# Obtener la ruta actual
currentPath = os.getcwd()
currentPath

'c:\\proyectos\\proyectos GitHub\\SkillMatchCV\\CreadorDB'

In [9]:
# Directorio que contiene los archivos a cargar, y directorio que contiene los archivos cargados
documentsUploadPath = os.path.join(currentPath, 'carga-documentos')
loadedDocumentsPath = os.path.join(documentsUploadPath, 'documentos-cargados')

In [11]:
user = ""
validOptions = ["usuario1", "usuario2", "usuario3", "usuario4", "usuario5"]
while user not in validOptions:
    user = input('¿Quién es el usuario que carga los documentos? Escriba un nombre válido: ("usuario1", "usuario2", "usuario3", "usuario4" "usuario5): ')

password = ''
while password != user:
    password = input('Escriba la contraseña para el usuario indicado: ')

category = ""
validOptions = ["cv", "texto", "otro"]
while category not in validOptions:
    category = input('Elija la categoría de los documentos a cargar ("cv", "texto", "otro"): ')


# Cargar el DataFrame si ya existe
if os.path.exists('documentosJP_df.csv'):
    df = pd.read_csv('documentosJP_df.csv')
else:
    # Si no existe, crea un DataFrame vacío
    df = pd.DataFrame(columns=['category', 'dateYear', 'dateMonth', 'dateDay', 'format', 'name', 'path', 'text', 'timeHour', 'timeMinutes', 'timeSeconds', 'user'])

# Acceder a los registros de Pinecone para verificar archivos cargados
records = Pinecone.from_existing_index(indexName, embeddings)

# Mientras exista un archivo en la carpeta de cargas
while len(os.listdir(documentsUploadPath)) > 1:
  # Recorrer los archivos en el directorio carga-documentos
  for file in os.listdir(documentsUploadPath):

      if file.endswith((".txt", ".docx", ".pdf")):


          # Si el archivo no está cargado en Pinecone, se extrae el texto, los vectores y los metadatos, y se carga en df y en Pinecone
          record = records.similarity_search(query='a', k=1, filter= {"name" : {"$eq": f"{file}"}})
          if len(record) == 0:
            fullPath = os.path.join(documentsUploadPath, file)
            try:
                if file.endswith(".txt"):
                  format = ".txt"
                  loader = TextLoader(fullPath, encoding='UTF-8')
                elif file.endswith(".docx"):
                  format = ".docx"
                  loader = Docx2txtLoader(fullPath)
                elif file.endswith(".pdf"):
                  format = ".pdf"
                  loader = PyPDFLoader(fullPath)              
                else:
                  format = 'Desconocido'
                  os.remove(fullPath)

                # Metadatos
                currentDate = date.today()
                # Obtener los componentes de la fecha (año, mes y día) como enteros
                dateYear = int(currentDate.year)
                dateMonth = int(currentDate.month)
                dateDay = int(currentDate.day)

                currentTime = datetime.datetime.now().time()
                # Obtener los componentes de la hora (hora y minutos) como enteros
                timeHour = int(currentTime.hour)
                timeMinutes = int(currentTime.minute)
                timeSeconds = int(currentTime.second)

                # Carga en Pinecone
                data = loader.load()
                #docs=textSplitter.split_documents(data)
                docs = data
                for i in range(len(docs)):
                    #docs[i].page_content = docs[i].page_content.replace('\n', ' ')
                    docSearch = Pinecone.from_texts([docs[i].page_content], embeddings, index_name=indexName,
                                                    metadatas = [{
                                                            'category': category,
                                                            'chunk_n': i+1,
                                                            'chunk_size': len(docs[i].page_content),
                                                            'chunks_total': len(docs),
                                                            'dateYear': dateYear,
                                                            'dateMonth': dateMonth,
                                                            'dateDay': dateDay,
                                                            'format': format,
                                                            'name': file,
                                                            'path': fullPath,
                                                            #'text': docs[i].page_content.replace('\n', ' '), # innecesario: 'text' se carga por defecto
                                                            'timeHour': timeHour,
                                                            'timeMinutes': timeMinutes,
                                                            'timeSeconds': timeSeconds,
                                                            'user': user
                                                        }])

                # Extracción de texto limpio (data es una lista de tuplas: [(page_content, metadata)])
                texto = ''
                for i in range(len(data)):
                    texto += ' ' + data[i].page_content.replace('\n', ' ')

                # Agregar nueva fila al DataFrame
                newRow = {
                            'category': category,
                            'dateYear': dateYear,
                            'dateMonth': dateMonth,
                            'dateDay': dateDay,
                            'format': format,
                            'name': file,
                            'path': fullPath,
                            'text': texto,
                            'timeHour': timeHour,
                            'timeMinutes': timeMinutes,
                            'timeSeconds': timeSeconds,
                            'user': user
                            }
                df2 = pd.DataFrame([newRow])
                df = pd.concat([df, df2], ignore_index=True)

                # Mueve el archivo a carpeta documentos-cargados
                shutil.move(fullPath, loadedDocumentsPath)
                print(f'el archivo {file} fue cargado en Pinecone correctamente')
            except:
              os.remove(fullPath)
              print(f'No se pudo acceder al contenido del archivo {file}')
              


          else:
            # El archivo ya ha sido cargado, eliminarlo del directorio carga-documentos
            fullPath = os.path.join(documentsUploadPath, file)
            os.remove(fullPath)
            print(f'el archivo {file} ya se encontraba cargado en Pinecone')

# Guardar el DataFrame en un archivo CSV
df.to_csv('documentosJP_df.csv', index=False)

el archivo 100 Joshua Ríos.txt ya se encontraba cargado en Pinecone
el archivo 101 Ana Belén González.txt fue cargado en Pinecone correctamente
el archivo 102 Aldo Manuel Gray.txt fue cargado en Pinecone correctamente
el archivo 103 Moisés Ortiz.txt fue cargado en Pinecone correctamente
el archivo 104 Barbara Santos González.txt fue cargado en Pinecone correctamente
el archivo 105 Jonathan Pezo.txt fue cargado en Pinecone correctamente
el archivo 106 Nicole Hodge.txt fue cargado en Pinecone correctamente
el archivo 107 Amanda Villalobos.txt fue cargado en Pinecone correctamente
el archivo 108 Fernando Craig.txt fue cargado en Pinecone correctamente
el archivo 109 Diane Espinoza.txt fue cargado en Pinecone correctamente
el archivo 110 Noelia Ruiz.txt fue cargado en Pinecone correctamente
el archivo 111 Cristina Villanueva.txt fue cargado en Pinecone correctamente
el archivo 112 Juan David Mendez.txt fue cargado en Pinecone correctamente
el archivo 113 Nicolás Campbell.txt fue cargado en

In [13]:
df.tail()

,category,dateYear,dateMonth,dateDay,format,name,path,text,timeHour,timeMinutes,timeSeconds,user
195,cv,2023,10,25,.txt,196 Adriana Zapata.txt,c:\proyectos\proyectos GitHub\SkillMatchCV\Cre...,Nombre: Adriana Zapata Correo Electrónico: a...,18,9,45,usuario1
196,cv,2023,10,25,.txt,197 Rebecca Campbell.txt,c:\proyectos\proyectos GitHub\SkillMatchCV\Cre...,Nombre: Rebecca Campbell Correo Electrónico:...,18,9,47,usuario1
197,cv,2023,10,25,.txt,198 Brandi San Martín.txt,c:\proyectos\proyectos GitHub\SkillMatchCV\Cre...,Nombre: Brandi San Martín Correo Electrónico...,18,9,48,usuario1
198,cv,2023,10,25,.txt,199 Miguel Mendoza.txt,c:\proyectos\proyectos GitHub\SkillMatchCV\Cre...,Nombre: Miguel Mendoza Correo Electrónico: m...,18,9,50,usuario1
199,cv,2023,10,25,.txt,200 Donald Hester.txt,c:\proyectos\proyectos GitHub\SkillMatchCV\Cre...,Nombre: Donald Hester Correo Electrónico: do...,18,9,52,usuario1


In [14]:
docSearch = Pinecone.from_existing_index(indexName, embeddings)

In [15]:
type(docSearch)

langchain.vectorstores.pinecone.Pinecone

In [ ]:
"""
The metadata filters can be combined with AND and OR:

$eq - Equal to (number, string, boolean)
$ne - Not equal to (number, string, boolean)
$gt - Greater than (number)
$gte - Greater than or equal to (number)
$lt - Less than (number)
$lte - Less than or equal to (number)
$in - In array (string or number)
$nin - Not in array (string or number)
"""

In [ ]:
k = 5
experiencia = 'Data Science y Machine Learning'
herramientas = 'TensorFlow, Keras, Python, PyTorch, Pandas'
idiomas = 'inglés y francés'

In [ ]:
k = 5
experiencia = 'desarrollo web'
herramientas = 'Django, Streamlit, Python, Java'

In [ ]:
k = 5
experiencia = 'finanzas y negocios'
herramientas = 'Excel, Office y NetSuite'

In [16]:
k = 5
experiencia = 'recursos humanos y contratación de personal'
herramientas = 'TalentSoft y SmartRecruiters'

In [19]:
#query= f'Quiero traer los {k} currículums que tengan más experiencia en {experiencia} y utilicen las siguientes herramientas: {herramientas} y con los siguientes idiomas: {idiomas}'
query = f' {experiencia} {herramientas}'
docs=docSearch.similarity_search(query, k=k, filter=
                                   {
                                    #'category': {'$eq': str},
                                    #'chunk_n': {'$gte': int},
                                    #'chunk_size': {'$gte': int},
                                    #'chunks_total': {'$gte': int},
                                    #'dateYear': {'$gte': str},
                                    #'dateMonth': {'$gte': str},
                                    #'dateDay': {'$gte': str},
                                    #'format': {'$eq': str},
                                    #'name': {'$eq': str},
                                    #'path': {'$eq': str},
                                    #'timeHour': {'$eq': str},
                                    #'timeMinutes': {'$eq': str},
                                    #'timeSeconds': {'$eq': str},
                                    #'user': {'$eq': str}
                                    })

In [20]:
for i in range(len(docs)):
    print(f'Documento {i+1}: ')
    print(docs[i].metadata['name'])
    print('')
    print(docs[i].page_content)
    print('')
    

Documento 1: 
180 Roberto José Verdugo.txt

Nombre: Roberto José Verdugo

Correo Electrónico: robertojoseverdugo@skillmatchcv.com
 
About:
Profesional con background en Evaluación de Recursos Humanos.
Me destaco por mi Planificación Financiera.
 
Herramientas técnicas:
1. Glassdoor for Employers
2. PeopleSoft
3. Job Boards (e.g., Indeed, Glassdoor)
4. Crelate
5. Taleo

Experiencia laboral:
1. Especialista en Cultura Organizacional en AutoRevolution Automotive (7 años)
2. Director de Recursos Humanos en Creative Minds Academy (6 años)

Idiomas:
1. Español (nativo)

Documento 2: 
80 Isabella Michael.txt

Nombre: Isabella Michael

Correo Electrónico: isabellamichael@skillmatchcv.com
 
About:
Profesional con background en Selección de Talento.
Me destaco por mi Honestidad.
 
Herramientas técnicas:
1. Workable
2. Namely
3. Employee Wellness Programs
4. CareerBuilder for Employers
5. Talentsoft
6. Interview Scheduling Software
7. Workforce Planning Tools
8. SAP SuccessFactors
9. Lever

Exper